In [4]:
import numpy as np 
import os 
from  scipy.io.arff import loadarff
import pandas as pd

# Loading Data

In [5]:
# first lets read the file and conver the string to an actual string by decoding it 
#loading a dataset
from sklearn.preprocessing import OneHotEncoder
def onehotencode(x):
    #instantiate
    ohe = OneHotEncoder()
    ohe.fit(x)
    ohe_trans = ohe.transform(x).toarray()
    return ohe_trans
    
def load_dataset(FILE_NAME):
    data,meta = loadarff(FILE_NAME)
    df = pd.DataFrame(data)
    cleaner = lambda x: x.decode('utf-8')
    vcleaner = np.vectorize(cleaner)
    to_clean = lambda x: vcleaner(x)
    vto_clean = np.vectorize(to_clean)
    cleaned_df = df.apply(vto_clean)
    return cleaned_df

# this will get you the testing and training data from the specified folder 
def get_working_files(NAME):
    '''
    Given the name of the folder, it will return a list of filenames corresponding to the training and testing
    '''
    training_list = []
    testing_list = []
    for file in os.listdir(f"./data/{NAME}"):
        if 'train' in file: 
            training_list.append(file)
        else: 
            testing_list.append(file)
    #make sure we have the same number of Training & Testing Samples 
    assert len(testing_list) == len(training_list)
    print('Training == Testing')
    return sorted(training_list), sorted(testing_list)

# this will load the batches into memory 
def load_batches(FolderName, TRAIN_TEST_LIST,label=None):
    PATH = f'./data/{FolderName}/'
    MatrixList = []
    for idx, batch in enumerate(TRAIN_TEST_LIST): 
        path_to_load = PATH + batch
        MATRIX = load_dataset(path_to_load)
        MatrixList.append(MATRIX.values)
        if label == 'Train':
            print(f"The number of training instances in batch {idx+1} is: {MATRIX.shape[0]}")
        else:
            print(f"The number of testing instances in batch {idx+1} is: {MATRIX.shape[0]}")
    return MatrixList



In [6]:
datasets = ['kropt','soybean','satimge']

train_list, test_list = get_working_files(datasets[0])

train_matrix = load_batches(datasets[0],train_list,label='Train')

#test_matrix = load_batches(datasets[0],train_list,label='Testing')

Training == Testing
The number of training instances in batch 1 is: 25257
The number of training instances in batch 2 is: 25253
The number of training instances in batch 3 is: 25249
The number of training instances in batch 4 is: 25251
The number of training instances in batch 5 is: 25246
The number of training instances in batch 6 is: 25250


KeyboardInterrupt: 

In [3]:
PATH = './data/kropt'
file_name = 'kropt.fold.000000.test.arff'
df = load_dataset(PATH + '/' + file_name)
#sorted(os.listdir("./data/kropt"))
df.head()

,white_king_col,white_king_row,white_rook_col,white_rook_row,black_king_col,black_king_row,game
0,d,2,a,3,g,4,thirteen
1,c,2,h,7,d,5,fourteen
2,b,2,d,1,g,2,eleven
3,d,3,e,2,b,2,six
4,d,3,a,2,d,1,five


In [11]:
from sklearn.preprocessing import OneHotEncoder
X = df['white_king_col']
X = np.array(X).reshape(-1,1)
ohe = OneHotEncoder()
ohe.fit(X)
data = ohe.transform(X).toarray()
data

array([[0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       ...,
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.]])

In [96]:
test_col = df['white_king_col']
test_col = np.array(test_col).reshape(-1, 1)
kk = onehotencode(test_col)

AttributeError: 'OneHotEncoder' object has no attribute 'ndim'

In [72]:
# I need to split up the datasets 
# when I load them, they need to be in the form of X_train, X_test, y_train, y_test = load_batches()
# I need a training phase and a prediction phase 
# training phase just classifies them and then we evaluate with the predict() 

In [73]:
def similarity(x, y):
    '''
    From the paper Instance Based Learning: 
    Similarity(x, y) = - sqrt(summation(f(xi, yi)))
    
    For numeric:
    Euclidean distance 
    f(xi, yi) = (xi - yi)**2
    
    For boolean and symbolic 
    are they equal or not
    f(xi, yi) = (xi =! yi)
    
    if both attributes are missing 
    f(xi, yi) = 1
    
    
    '''
    return -1 * np.sqrt(np.sum((x-y)**2))
    
    

In [78]:
test_matrix

[array([['b', '1', 'h', ..., 'e', '3', 'fifteen'],
        ['d', '3', 'f', ..., 'b', '8', 'eight'],
        ['d', '1', 'e', ..., 'b', '6', 'thirteen'],
        ...,
        ['c', '2', 'd', ..., 'e', '5', 'fourteen'],
        ['b', '1', 'd', ..., 'c', '7', 'eleven'],
        ['c', '2', 'e', ..., 'c', '4', 'thirteen']], dtype=object),
 array([['d', '2', 'a', ..., 'g', '4', 'thirteen'],
        ['c', '2', 'h', ..., 'd', '5', 'fourteen'],
        ['b', '2', 'd', ..., 'g', '2', 'eleven'],
        ...,
        ['c', '2', 'd', ..., 'e', '5', 'fourteen'],
        ['b', '1', 'd', ..., 'c', '7', 'eleven'],
        ['c', '2', 'e', ..., 'c', '4', 'thirteen']], dtype=object),
 array([['d', '2', 'a', ..., 'g', '4', 'thirteen'],
        ['c', '2', 'h', ..., 'd', '5', 'fourteen'],
        ['b', '2', 'd', ..., 'g', '2', 'eleven'],
        ...,
        ['c', '2', 'd', ..., 'e', '5', 'fourteen'],
        ['b', '1', 'd', ..., 'c', '7', 'eleven'],
        ['c', '2', 'e', ..., 'c', '4', 'thirteen']], dtype=